# Attention in Transformers: Concepts and Code in PyTorch
Attention mecahnism is a key breakthorugh in transformer development. 

Ref: 
1. Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio. "Neural machine translation by jointly learning to align and translate." arXiv preprint arXiv:1409.0473 (2014).
2. Luong, Minh-Thang, Hieu Pham, and Christopher D. Manning. "Effective approaches to attention-based neural machine translation." arXiv preprint arXiv:1508.04025 (2015).
3. Vaswani, Ashish, et al. "Attention is all you need." Advances in neural information processing systems 30 (2017).

Machine Translation:

encoder decoder mechanism works better

# Attention in transformers: Concept and code in PyTorh
## The main idea behind Transformers and attention

lanuage models like chatgpt is based on transformers. Transformers can look complicated, but fundamanetally they require 3 main parts.
1. word embedding: Converts inputs into numbers. Converts words, bits of words and symbols , collectively called __Tokens__, into numbers. We need this because Transformers are a type of Neural Networkd and NN only have numbers for input values. For example: if the input is "" Tell me about Pizza!", than word embedding will convert it into numbers $[0.5 1.3 2.03 -0.45!]$
2. Positional Encodeing: Helps keep track of word order. Squatch eats pizza or pizza eat Squatch.
3. Attention: Helps establish realtionships among words. Transformer establishes realtionships among words with Attention. Example, "The Pizza came out of the oven and it tasted good. The word 'it' can refer to pizza or oven" Thats why transformer correctly associate the word with Pizza not with the oven. Transformers have something called Attention, which is a mecahnism to correclty assocaite the word 'it' witht he word 'pizza'. There are differnet types of attention,
    - __Self Attention:__ self attention works by seeing how similar each word is to all of the words in the sentens, inclsing itself. Self attention calcualtes the similarities for every word in the sentense. Once the similarities are calcualted they are used to determine how the __Transformer__ encode each word.
      $$ Attention(Q,K,V)=SoftMax \bigg(\frac{QK^T}{\sqrt{d_k}}\bigg)V$$
    - Masked-Slef-attention: $$ Masked-Slef-attention (Q,K,V,M)=SoftMax \bigg(\frac{QK^T}{\sqrt{d_k}}+ M\bigg)V$$
    - Multi-Head atention: when we have multi attetnion heads
    - Encoder-decoder attention / cross-atetnion: example Seq2Seq model
  
## The matrix math for calcualting self-attention
At first glance, the equation for calculating Self-Atention can be a little intimidating ... 
$$ Attention(Q,K,V)=SoftMax \bigg(\frac{QK^T}{\sqrt{d_k}}\bigg)V$$
The term query, key and value comes from database terminoloigy. Example, lets look the database. if the database has lastname and room number of a hotel. If a search a name from the database than this serach anme is query and the stored values in the databaser is called key. In database terminology room numbers are values.
- $Q$ stands for query: It is the things we use to search the database.
- $K$ stands for Key. The computer calculates the similarity between query and all the keys and return the ranks
- $V$ stands for Value. The values are what the database turns results of the search.
- $d_k$ is the dimension of the key matrix and dimension refers to number of values we have for each token, 
### How we determine the Queries, Keys, and Values in the context of transformer.
- Rememeber that self-attention alcualtes similarity between each word and itself and all of the other words. that means we need to calcualte query and key for each word and just like we saw in the database example, each key needs to return a value. Example. Lets assume we write in the prompt "Write a poem"
    - First things transfromer do with these words . It woill convert into word embedding. Than transformer adds positional encodeing to the word embedding to get these numbers or encodings that represent each word in the prompt. NOTE: in this simple example , we're just going to use 2 numbers to reapresent each word in the prompt. However, it's much more common to use 512 or more numbers to represent each word.
    Note that I have put the weights as transpose. This is because PyTorch prints out the weights in a way that requires them to be transposed before we van get the math to work out correctly.
- dot products (unscaled) can be used as an unscalaed measure of similarity between two things, and this metric is cloasely realted to something called __Cosine Similarity__. The big difference is that the cosine similarity scales Dot Product to be between $-1$ and $1$. In contrast the dot prodcut similarity isn't scaled.
- once we divide by $d_k$ , we get scaled dot-product similarity. Note: saling by just $d_k$ of the values per token doesn't scale dot product similarities in any kind of systematic way. Thats said, even with this limited scaling the original authors of the transformer said that it improved perfromance.


In sumaary, the equation for self attention does is calclcualte the scaled dot-proidcut similarieties among all of the words, convert those scaled similarities into percentages with the `SoftMax()` fucniton and then use those percentages to sclae the values to becaome the self-attention scalere for each word.

---- 

In this tutorial, we will code **Self-Attention** in **[PyTorch](https://pytorch.org/)**. **Attention** is an essential component of neural network **Transformers**, which are driving the current excitement in **Large Language Models** and **AI**. Specifically, an **Enecoder-Only Transformer**, illustrated below, is the foundation for the popular model **BERT**. 

<img src="./images/encoder_only_1.png" alt="an enecoder-only transformer neural network" style="width: 800px;">

At the heart of **BERT** is **Self-Attention**, which allows it to establish relationships among the words, characters and symbols, that are used for input and collectively called **Tokens**. For example, in the illustration below, where the word **it** could potentially refer to either **pizza** or **oven**, **Attention** could help a **Transformer** establish the correctly relationship between the word **it** and **pizza**.

<img src="./images/attention_ex_1.png" alt="an illustration of how attention works" style="width: 800px;"/>

In this tutorial, you will...

- **[Code a Basic Self-Attention Class!!!](#selfAttention)** The basic self-attention class allows the transformer to establish relationships among words and tokens.

- **[Calculate Self-Attention Values!!!](#calculate)** We'll then use the class that we created, SelfAttention, to calculate self-attention values for some sample data.
 
- **[Verify The Calculations!!!](#validate)** Lastly, we'll validate the calculations made by the SelfAttention class..


# Import the modules that will do all the work


In [1]:
import torch ## torch let's us create tensors and also provides helper functions
import torch.nn as nn ## torch.nn gives us nn.module() and nn.Linear()
import torch.nn.functional as F # This gives us the softmax()

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

# Code Self-Attention
<a id="selfAttention"></a>

We will use `d_model` to define the size of weigths matrices that we'll use to create the Querries, Keys and Values. If `d_model=2` that means we have two embedding numbers per token

In [ ]:
class SelfAttention(nn.Module): 
                            
    def __init__(self, d_model=2,  
                 row_dim=0, 
                 col_dim=1):
        ## d_model = the number of embedding values per token.
        ##           Because we want to be able to do the math by hand, we've
        ##           the default value for d_model=2.
        ##           However, in "Attention Is All You Need" d_model=512
        ##
        ## row_dim, col_dim = the indices we should use to access rows or columns

        
        super().__init__() # call the parent's __init__ method otherwise there is no point in inheriti from a class to begin with.
        
        ## Initialize the Weights (W) that we'll use to create the
        ## query (q), key (k) and value (v) for each token
        ## NOTE: A lot of implementations include bias terms when
        ##       creating the the queries, keys, and values, but
        ##       the original manuscript that described Attention,
        ##       "Attention Is All You Need" did not, so we won't either
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False) # create weight matrix
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        
        self.row_dim = row_dim
        self.col_dim = col_dim

    # the forward methos where we actually calculate the self-attention values for each tokens.
    def forward(self, token_encodings):
        ## Create the query, key and values using the encoding numbers
        ## associated with each token (token encodings)
        q = self.W_q(token_encodings)
        k = self.W_k(token_encodings)
        v = self.W_v(token_encodings)

        ## Compute similarities scores: (q * k^T)
        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        ## Scale the similarities by dividing by sqrt(k.col_dim)
        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

        ## Apply softmax to determine what percent of each tokens' value to
        ## use in the final attention values.
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)

        ## Scale the values by their associated percentages and add them up.
        attention_scores = torch.matmul(attention_percents, v)

        return attention_scores

# Calculate Self-Attention
<a id="calculate"></a>

In [ ]:
## create a matrix of token encodings...
encodings_matrix = torch.tensor([[1.16, 0.23],
                                 [0.57, 1.36],
                                 [4.41, -2.16]])

## set the seed for the random number generator
torch.manual_seed(42)

## create a basic self-attention ojbect
selfAttention = SelfAttention(d_model=2,
                               row_dim=0,
                               col_dim=1)

## calculate basic attention for the token encodings
selfAttention(encodings_matrix)

# Print Out Weights and Verify Calculations
<a id="validate"></a>

In [ ]:
## print out the weight matrix that creates the queries
selfAttention.W_q.weight.transpose(0, 1)

In [ ]:
## print out the weight matrix that creates the keys
selfAttention.W_k.weight.transpose(0, 1)

In [ ]:
## print out the weight matrix that creates the values
selfAttention.W_v.weight.transpose(0, 1)

In [ ]:
## calculate the queries
selfAttention.W_q(encodings_matrix)

In [ ]:
## calculate the keys
selfAttention.W_k(encodings_matrix)

In [ ]:
## calculate the values
selfAttention.W_v(encodings_matrix)

In [ ]:
q = selfAttention.W_q(encodings_matrix)
q

In [ ]:
k = selfAttention.W_k(encodings_matrix)
k

In [ ]:
sims = torch.matmul(q, k.transpose(dim0=0, dim1=1))
sims

In [ ]:
scaled_sims = sims / (torch.tensor(2)**0.5)
scaled_sims

In [ ]:
attention_percents = F.softmax(scaled_sims, dim=1)
attention_percents

In [ ]:
torch.matmul(attention_percents, selfAttention.W_v(encodings_matrix))